# Script to extract arrival times from raw THz streaking data at SwissFEL Alvra and save results as h5 file
- General documentation https://docs.google.com/document/d/1Ll3_WsHzFwuvL-sWgqWQUSYKJRgFG6W7PWY-NJ40WS0/edit?usp=sharing
- User guide ADD WEBSITE
- Comments to christopher.arrell@psi.ch

In [17]:
import photodiag
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import json
import os
from IPython.display import clear_output, display

In [18]:
def LoadData(fn):
    with h5.File(fn, 'r') as f:
        try:
            TOF = -f['data/SAROP11-PALMK118:CH2_BUFFER/data'][:]
            uTOF = -f['data/SAROP11-PALMK118:CH1_BUFFER/data'][:]
            PulseIds = f['data/SAROP11-PALMK118:CH2_BUFFER/pulse_id'][:]
            Events = f['data/SAR-CVME-TIFALL5:EvtSet/data'][:]
        except:
            TOF = -f['SAROP11-PALMK118:CH2_BUFFER/data'][:]
            uTOF = -f['SAROP11-PALMK118:CH1_BUFFER/data'][:]
            PulseIds = f['SAROP11-PALMK118:CH2_BUFFER/pulse_id'][:]
            Events = f['SAR-CVME-TIFALL5:EvtSet/data'][:]
            
        FEL = Events[:,48]
        Laser = Events[:,18]
        Darkshot = Events[:,21]
        index_pump = np.logical_and.reduce((FEL, Laser, np.logical_not(Darkshot)))
    return(TOF[index_pump], uTOF[index_pump], PulseIds[index_pump])

def Time2t0(dataIn):
    #for stages in S
    dataIn -= dataIn.mean()
    return dataIn*1e15


### User inputs


In [19]:
energyFrom = 1300
energyTo = 1800
energySteps = 1000
ScanFn = '/sf/alvra/data/p17589/res/scan_info/Febpy_timescan_7154eV_150mJ_006_scan_info.json'
CalibrationFn = f'/sf/alvra/data/p17589/res/PhotoDiag/scan_info/2019-07-05_10:44:32.palm_etof'
Datafn = ScanFn[36:-5]+'_TT_data'
Datafn_pids = ScanFn[36:-5]+'_TT_data_pids'

### Setup and load data

In [20]:
palm = photodiag.PalmSetup(
    channels={'0': 'SAROP11-PALMK118:CH1_BUFFER', '1': 'SAROP11-PALMK118:CH2_BUFFER'},
    noise_range=[0, 250], energy_range=np.linspace(energyFrom, energyTo, energySteps),
)
palm.load_etof_calib(CalibrationFn)


with open(ScanFn) as f:
    data = json.load(f)
numFiles = len(data['scan_files'])
StagePOS = data['scan_values'][:]

wf_str = []
wf_ref = []
pids = []

for i in range(0,numFiles):
    clear_output(wait=True)
    print('File loaded: \n' + data['scan_files'][i][0])
    (tmpwf_str, tmpwf_ref, tmp_pids) = LoadData(str(data['scan_files'][i][0]))
    
    wf_str.append(tmpwf_str)
    wf_ref.append(tmpwf_ref)
    pids.append(tmp_pids)
    
wf_str = np.asarray(wf_str)
wf_ref = np.asarray(wf_ref)
pids = np.asarray(pids)

StagePOS = np.asarray(StagePOS)

TimeAx = Time2t0(StagePOS)
StagePOS = np.asarray(StagePOS)

File loaded: 
/sf/alvra/data/p17589/raw/scan_data/Febpy_timescan_7154eV_150mJ_006/Febpy_timescan_7154eV_150mJ_006_step0053.BSREAD.h5


### Extract arrival times and save

In [21]:
calibPALM = -.08 # ev/fs  This from THz streak plot script

In [22]:
Delays = []
for i in range(0,len(wf_str)):
    delays, pulse_lengths = palm.process({'0': wf_ref[i], '1': wf_str[i]}, noise_thr=0, jacobian=False, peak='max')
    Delays.append(delays)
DelaysRAW = np.asarray(Delays)
Delays = np.asarray(Delays)/calibPALM

In [27]:
SaveFn = '/das/work/units/alvra/p17589/PhotonDiag/'+Datafn+'.h5'
with h5.File(SaveFn,'w-') as fh:
    fh['Delays_fs'] = Delays
    fh['Delays_eV'] = DelaysRAW
    fh['calibPALM'] = calibPALM
    fh['PulseIDs'] = pids
    fh['Calib_fn'] = CalibrationFn
    fh['Scan_fn'] = ScanFn
    fh['Energy vals'] = 'Energy from '+str(energyFrom)+ 'Energy to '+str(energyTo)+ 'Energy steps '+str(energySteps)  